### 目的
- 学習後の重みについて可視化する
- drop_out
- l1ノルム追加時の精度と重みについて可視化する

In [96]:
!pwd

/Users/shibuya/NN_check


In [32]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import os
from keras import backend as K





def set_seed(seed = 42):
    
    tf.random.set_seed(seed)
    #session_conf = tf.ConfigProto(
    #intra_op_parallelism_threads=1,
    #inter_op_parallelism_threads=1
    #)
    #sess = tf.Session(graph=tf.get_default_graph())
    #sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    #K.set_session(sess)
    np.random.seed(seed)
    # for built-in random
    random.seed(seed)

In [33]:
import keras
from keras.datasets import mnist
from keras import regularizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam

# MNISTデータを読込む
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# MNISTデータを加工する
x_train  = x_train.reshape(60000, 784)
x_test   = x_test.reshape(10000, 784)
x_train  = x_train.astype('float32')
x_test   = x_test.astype('float32')
x_train /= 255
x_test  /= 255
y_train  = keras.utils.to_categorical(y_train, 10)
y_test   = keras.utils.to_categorical(y_test, 10)



# モデルの構築
inputs = Input(shape=(784,))

set_seed(42)
x = Dense(784, activation='relu',kernel_initializer='random_uniform', name = "hoge")(inputs)
x = Dense(100, activation='relu',kernel_initializer='random_uniform')(inputs)
x = Dense(10, activation='softmax', kernel_initializer='random_uniform')(x)
#x4 = x2 + x3
model = Model(inputs=inputs, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 学習
epochs = 1
batch_size = 100
set_seed(42)
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# 検証
score = model.evaluate(x_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.2373 - accuracy: 0.9306

Test loss: 0.23732763528823853
Test accuracy: 0.9305999875068665


In [35]:
set_seed(42)
x = Dense(784, activation='relu',kernel_initializer='random_uniform', name = "hoge")(inputs)
x = Dense(100, activation='relu',kernel_initializer='random_uniform')(inputs)
x = Dense(10, activation='softmax', kernel_initializer='random_uniform')(x)
model2 = Model(inputs=inputs, outputs=x)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 学習
epochs = 1
batch_size = 100
set_seed(42)
history = model2.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# 検証
score = model2.evaluate(x_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.2373 - accuracy: 0.9306

Test loss: 0.23732763528823853
Test accuracy: 0.9305999875068665


## class model

In [26]:
inputs = Input(shape=(784,))

set_seed(42)
#x = Dense(784, activation='relu',kernel_initializer='random_uniform', name = "hoge")(inputs)
#x = Dense(100, activation='relu',kernel_initializer='random_uniform')(inputs)
#x = Dense(10, activation='softmax', kernel_initializer='random_uniform')(x)


# Define the subclassed model.
class Model_class(Model):
    def __init__(self):
        super(Model_class, self).__init__()
        set_seed(42)
        self.layer1 = Dense(784, activation='relu',kernel_initializer='random_uniform', name = "hoge")
        self.layer2 = Dense(100, activation='relu',kernel_initializer='random_uniform')
        self.layer3 = Dense(10, activation='softmax', kernel_initializer='random_uniform')
        

    def call(self, inputs):
        x = self.layer1(inputs)
        x = self.layer2(x)
        return self.layer3(x)

model = Model_class()
# Call model on inputs to create the variables of the dense layer.
_ = model(inputs)
#x4 = x2 + x3
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 1
batch_size = 100
set_seed(42)
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

# 検証
score = model.evaluate(x_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
600/600 [==============================] - 5s 7ms/step - loss: 0.5325 - accuracy: 0.8527 - val_loss: 0.1171 - val_accuracy: 0.9642
Epoch 2/2
313/313 [==============================] - 1s 3ms/step - loss: 0.0842 - accuracy: 0.9736

Test loss: 0.08421523123979568
Test accuracy: 0.9735999703407288


### 再現性の確認

In [23]:
inputs = Input(shape=(784,))

set_seed(42)
x = Dense(784, activation='relu',kernel_initializer='random_uniform', name = "hoge")(inputs)
x = Dense(100, activation='relu',kernel_initializer='random_uniform')(inputs)
x = Dense(10, activation='softmax', kernel_initializer='random_uniform')(x)
#x4 = x2 + x3
model = Model(inputs = inputs, outputs=x)

In [18]:
model2 = Model_class()
_ = model2(inputs)

In [21]:
model.layers[1].get_weights()[0]  == model2.layers[1].get_weights()[0]

array([[ 0.01878912, -0.00155212,  0.04309944, ...,  0.02197267,
        -0.04005278,  0.04480752],
       [-0.00537275, -0.0163908 , -0.00531293, ..., -0.02761691,
        -0.02099543, -0.04422212],
       [ 0.03057282,  0.04772124,  0.0273265 , ...,  0.01920762,
        -0.01681979,  0.04212563],
       ...,
       [ 0.0177031 ,  0.00048262, -0.03944298, ...,  0.0383727 ,
        -0.01793129, -0.02333527],
       [ 0.03191893,  0.01330081,  0.03970552, ..., -0.03449966,
        -0.01427802,  0.02485213],
       [-0.02796367, -0.00531797, -0.01274946, ...,  0.02140654,
         0.00115863,  0.01362861]], dtype=float32)

array([[ 0.01878912, -0.00155212,  0.04309944, ...,  0.02197267,
        -0.04005278,  0.04480752],
       [-0.00537275, -0.0163908 , -0.00531293, ..., -0.02761691,
        -0.02099543, -0.04422212],
       [ 0.03057282,  0.04772124,  0.0273265 , ...,  0.01920762,
        -0.01681979,  0.04212563],
       ...,
       [ 0.0177031 ,  0.00048262, -0.03944298, ...,  0.0383727 ,
        -0.01793129, -0.02333527],
       [ 0.03191893,  0.01330081,  0.03970552, ..., -0.03449966,
        -0.01427802,  0.02485213],
       [-0.02796367, -0.00531797, -0.01274946, ...,  0.02140654,
         0.00115863,  0.01362861]], dtype=float32)

In [91]:
w1_normal, w2_normal = model.layers[1].get_weights()[0],model.layers[2].get_weights()[0]

In [98]:
#for w in w1_normal:
#    print(w.shape)